# Bloodhound Playbook: Kerberos Delegation

In this playbook I will try to see if there is any (un)constrained delegation possibilities

## Connect to DB

In [1]:
from py2neo import Graph
g = Graph("bolt://127.0.0.1:7687", auth=("neo4j", "neo4j"))

### Count all computers with unconstraineddelegation

In [2]:
all_uncon_comp = g.run("""
MATCH (c:Computer {unconstraineddelegation:true})
RETURN COUNT (c)
""").to_data_frame()

In [ ]:
all_uncon_comp

### How many unconstrained delegation systems that are not part of the domain controllers group

In [4]:
not_dc_grp = g.run("""
MATCH (c1:Computer)-[:MemberOf*1..]->(g:Group)
WHERE g.objectsid ENDS WITH "-516"
WITH COLLECT(c1.name) AS domainControllers
MATCH (c2:Computer {unconstraineddelegation:true})
WHERE NOT c2.name IN domainControllers
RETURN c2.name,c2.operatingsystem,c2.serviceprincipalnames
ORDER BY c2.name ASC
""").to_data_frame()

In [ ]:
len(not_dc_grp)

In [6]:
not_dc_grp.columns = ['name','os','spn']

In [ ]:
#no_cluster[~no_cluster['spn'].str.contains("MSServerCluster", na=False)]
no_cluster = not_dc_grp.iloc[[0,1,2,3,4,32,41]]
no_cluster